In [ ]:
# Download the model
! python /mnt/lit-gpt/scripts/download.py --repo_id tiiuae/falcon-7b

In [ ]:
# Convert the checkpoint
! sudo python /mnt/lit-gpt/scripts/convert_hf_checkpoint.py --checkpoint_dir checkpoints/tiiuae/falcon-7b

In [ ]:
# Prepare the instruction dataset for training
! python /mnt/lit-gpt/scripts/prepare_alpaca.py \--checkpoint_dir checkpoints/tiiuae/falcon-7b/

In [ ]:
# Fine tune the model
! python /mnt/lit-gpt/finetune/lora.py \
  --checkpoint_dir checkpoints/tiiuae/falcon-7b/ \
  --precision bf16-true

In [6]:
# Generate output from the fine tuned model
! python /mnt/lit-gpt/generate/lora.py \
  --lora_path '/mnt/out/lora/alpaca/lit_model_lora_finetuned.pth' \
  --max_new_tokens 300 \
  --precision bf16-true

NOTE! Installing ujson may make loading annotations faster.
Loading model '/mnt/checkpoints/tiiuae/falcon-7b-instruct/lit_model.pth' with {'org': 'tiiuae', 'name': 'falcon-7b-instruct', 'block_size': 2048, 'vocab_size': 50254, 'padding_multiple': 512, 'padded_vocab_size': 65024, 'n_layer': 32, 'n_head': 71, 'n_embd': 4544, 'rotary_percentage': 1.0, 'parallel_residual': True, 'bias': False, 'n_query_groups': 1, 'shared_attention_norm': True, '_norm_class': 'LayerNorm', 'norm_eps': 1e-05, '_mlp_class': 'GptNeoxMLP', 'intermediate_size': 18176, 'condense_ratio': 1, 'r': 16, 'alpha': 32, 'dropout': 0.05, 'to_query': False, 'to_key': False, 'to_value': False, 'to_projection': False, 'to_mlp': False, 'to_head': False}
Time to instantiate model: 1.08 seconds.
Time to load the model weights: 6.01 seconds.
You (I) are chatting with a user R. Write a reply to their message.

### Your previous communication:


### Their new message:
What food do lamas eat?

### Your response:
Lamas, as intelligen

In [9]:
# Generate output from the base model that has not been fine tuned
! python /mnt/lit-gpt/generate/base.py --checkpoint_dir '/mnt/checkpoints/tiiuae/falcon-7b' \
    --prompt 'What food do lamas eat?' \
    --max_new_tokens 300 \
    --precision bf16-true

NOTE! Installing ujson may make loading annotations faster.
Loading model '/mnt/checkpoints/tiiuae/falcon-7b/lit_model.pth' with {'org': 'tiiuae', 'name': 'falcon-7b', 'block_size': 2048, 'vocab_size': 50254, 'padding_multiple': 512, 'padded_vocab_size': 65024, 'n_layer': 32, 'n_head': 71, 'n_embd': 4544, 'rotary_percentage': 1.0, 'parallel_residual': True, 'bias': False, 'n_query_groups': 1, 'shared_attention_norm': True, '_norm_class': 'LayerNorm', 'norm_eps': 1e-05, '_mlp_class': 'GptNeoxMLP', 'intermediate_size': 18176, 'condense_ratio': 1}
Time to instantiate model: 1.14 seconds.
Time to load the model weights: 6.08 seconds.
Global seed set to 1234
What food do lamas eat?
Lamas eat whatever their human hosts eat, but they are not allowed to eat meat and dairy products. The following is a list of some of the foods that Tibetan lamas eat: noodles, dairy products, eggs, meat and vegetables.[Note : This list is not comprehensive]
What is the diet of a monk?
A monk's diet consists main

In [7]:
# Quantize the base model and generate output
! python /mnt/lit-gpt/generate/base.py --quantize bnb.int8 \
    --checkpoint_dir 'checkpoints/tiiuae/falcon-7b' \
    --prompt 'What food do lamas eat?' \
    --precision bf16-true \
    --max_new_tokens 300


NOTE! Installing ujson may make loading annotations faster.
Loading model 'checkpoints/tiiuae/falcon-7b/lit_model.pth' with {'org': 'tiiuae', 'name': 'falcon-7b', 'block_size': 2048, 'vocab_size': 50254, 'padding_multiple': 512, 'padded_vocab_size': 65024, 'n_layer': 32, 'n_head': 71, 'n_embd': 4544, 'rotary_percentage': 1.0, 'parallel_residual': True, 'bias': False, 'n_query_groups': 1, 'shared_attention_norm': True, '_norm_class': 'LayerNorm', 'norm_eps': 1e-05, '_mlp_class': 'GptNeoxMLP', 'intermediate_size': 18176, 'condense_ratio': 1}
Time to instantiate model: 2.05 seconds.
Time to load the model weights: 7.69 seconds.
Global seed set to 1234
What food do lamas eat?
Lamas eat whatever their human caretakers eat. Unfortunately for the Tibetan Sheepdogs, the diet of lamas is not what they would prefer. It is thought that lamas eat one-third dairy products and two-thirds grains, although we have no way of knowing exactly what they eat.
How much does a lama eat?
According to the Lama

In [14]:
# Quantize the fine tuned model and generate output
! python /mnt/lit-gpt/generate/lora.py --quantize bnb.int8 \
    --checkpoint_dir 'checkpoints/tiiuae/falcon-7b' \
    --prompt 'What food do llamas eat?' \
    --precision bf16-true \
    --max_new_tokens 300

NOTE! Installing ujson may make loading annotations faster.
Loading model 'checkpoints/tiiuae/falcon-7b/lit_model.pth' with {'org': 'tiiuae', 'name': 'falcon-7b', 'block_size': 2048, 'vocab_size': 50254, 'padding_multiple': 512, 'padded_vocab_size': 65024, 'n_layer': 32, 'n_head': 71, 'n_embd': 4544, 'rotary_percentage': 1.0, 'parallel_residual': True, 'bias': False, 'n_query_groups': 1, 'shared_attention_norm': True, '_norm_class': 'LayerNorm', 'norm_eps': 1e-05, '_mlp_class': 'GptNeoxMLP', 'intermediate_size': 18176, 'condense_ratio': 1, 'r': 16, 'alpha': 32, 'dropout': 0.05, 'to_query': False, 'to_key': False, 'to_value': False, 'to_projection': False, 'to_mlp': False, 'to_head': False}
Time to instantiate model: 2.07 seconds.
Time to load the model weights: 7.57 seconds.
You (I) are chatting with a user R. Write a reply to their message.

### Your previous communication:


### Their new message:
What food do llamas eat?

### Your response:
I think the llamas eat grass!

### Their r